In [1]:
import pandas as pd
import numpy as np
import pandapower as pp
import pandapower.networks as nw
from pandapower.plotting.plotly.mapbox_plot import geo_data_to_latlong

In [2]:
testnet = nw.mv_oberrhein()
geo_data_to_latlong(testnet, projection='epsg:31467')

#testnet = pp.from_json('7068068.json')
#geo_data_to_latlong(testnet, projection='epsg:32632')

net = pp.to_json(testnet)

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)

c:\Users\Daniel\miniconda3\envs\TUM_Syngrid\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\Daniel\miniconda3\envs\TUM_Syngrid\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = 

In [3]:
line_std_properties = ["r_ohm_per_km", "x_ohm_per_km", "max_i_ka", "c_nf_per_km", "q_mm2", "type", "alpha"]
trafo_std_properties = ["sn_mva", "vn_hv_kv", "vn_lv_kv", "vk_percent", "vkr_percent", "pfe_kw", "i0_percent", "shift_degree", "tap_side", "tap_neutral", "tap_min", "tap_max", "tap_step_percent", "tap_step_degree", "tap_phase_shifter"]
trafo3w_std_properties = ["sn_hv_mva","sn_mv_mva","sn_lv_mva","vn_hv_kv", "vn_mv_kv","vn_lv_kv","vk_hv_percent", "vk_mv_percent","vk_lv_percent","vkr_hv_percent","vkr_mv_percent","vkr_lv_percent","pfe_kw","i0_percent","shift_mv_degree","shift_lv_degree","tap_side","tap_neutral","tap_min","tap_max","tap_step_percent"]

line_properties = ["name","from_bus", "to_bus","length_km", "r0_ohm_per_km", "x0_ohm_per_km", "c0_nf_per_km","df","g_us_per_km", "g0_us_per_km","parallel","max_loading_percent","temperature_degree_celsius", "tdpf","wind_speed_m_per_s", "wind_angle_degree", "conductor_outer_diameter_m", "air_temperature_degree_celsius","reference_temperature_degree_celsius", "solar_radiation_w_per_sq_m", "solar_absorptivity", "emissivity", "r_theta_kelvin_per_mw", "mc_joule_per_m_k", "std_type"]

ext_grid_properties = ["name", "bus", "vm_pu", "va_degree", "s_sc_max_mva", "s_sc_min_mva", "rx_max", "rx_min", "max_p_mw", "max_p_mw", "max_q_mvar", "min_q_mvar", "r0x0_max", "x0x_max", "slack_weight", "controllable", "in_service"]

bus_properties =  ["name","vn_kv","type","in_service", "max_vm_pu","min_vm_pu",]    
load_features = ['name', 'p_mw', 'q_mvar','max_p_mw', 'min_p_mw', 'max_q_mvar', 'min_q_mvar', 'const_z_percent', 'const_i_percent', 'sn_mva', 'scaling', 'in_service', 'type', 'controllable']
sgen_features = ['name', 'p_mw', 'q_mvar', 'max_p_mw', 'min_p_mw', 'max_q_mvar', 'min_q_mvar', 'sn_mva', 'scaling', 'in_service', 'type', 'current_source', 'k', 'rx', 'generator_type', 'lrc_pu', 'max_ik_ka', 'kappa', 'controllable']
switch_features = ['name', 'element', 'et', 'type', 'closed', 'z_ohm', 'in_ka']

trafo_properties = ["name", "hv_bus", "lv_bus", "vk0_percent", "vkr0_percent", "mag0_percent", "mag0_rx", "si0_hv_partial", "tap_pos", "in_service", "max_loading_percent", "parallel", "df", "tap_dependent_impedance", "vk_percent_characteristic", "vkr_percent_characteristic", "xn_ohm", "std_type"]


In [50]:
# if (typeof(input) == undefined) {
#     return {};
# }

# let input_data = JSON.parse(input['_object'])['data'];
# let input_indices = JSON.parse(input['_object'])['index'];
# let input_columns = JSON.parse(input['_object'])['columns'];
# let idx = input_columns.indexOf('bus', 0);

# let output = {};

# for (entry in input_data) {
#     if (input_data[entry][idx] == pointIndex) {
#         output.index = input_indices[entry];
#         for (property in properties) {
#             output[properties[property]] = (input_columns.indexOf(properties[property], 0) == -1) ? null : input_data[entry][input_columns.indexOf(properties[property], 0)];
#         }
#         break;
#     }
# }

# return output;

def extractPropertiesFromNet(input, index, properties):
    if input.empty:
        return {}
    
    output = {}

    for entry in input:
        output[entry] = input.T.loc[entry].iloc[0]
    
    return output

In [49]:
def createFeatures (isLines, ppdata, featureName, featureProperties, propertyGroupNames, propertyGroupFeatures):
    input_data = ppdata[featureName]
    input_geoCoords = pd.DataFrame()

    if featureName == 'bus' or featureName == 'line':
        input_geoCoords = ppdata[featureName + '_geodata']
        if featureName == 'bus':
            input_geoCoords = input_geoCoords.drop(['coords'], axis=1)

    else:
        input_geoCoords = ppdata.bus_geodata
        temp = pd.DataFrame()

        if featureName == 'ext_grid' :
            for bus in input_data['bus']:
                #print(input_geoCoords.T[bus].to_frame().T)
                ext_geocoords = input_geoCoords.T[bus].to_frame().T
                temp = pd.concat([temp, ext_geocoords], ignore_index=True)
            temp = temp.drop(['coords'], axis=1)

        elif featureName == 'trafo':
            i = 0
            for hv_bus, lv_bus in zip(input_data['hv_bus'], input_data['lv_bus']):
                hv_geocoords = input_geoCoords.T[hv_bus]
                lv_geocoords = input_geoCoords.T[lv_bus]
                tempLine = pd.Series(data={'hv_bus' : [hv_geocoords.x, hv_geocoords.y], 'lv_bus' : [lv_geocoords.x, lv_geocoords.y]}).to_frame().T
                temp = pd.concat([temp, tempLine], ignore_index=True)
                # hv_geocoords = input_geoCoords.T[hv_bus].to_frame().T
                # lv_geocoords = input_geoCoords.T[lv_bus].to_frame().T
                # hv_geocoords = hv_geocoords.rename(columns={'x' : 'x' + str(i), 'y' : 'y' + str(i)})
                # lv_geocoords = lv_geocoords.rename(columns={'x' : 'x' + str(i), 'y' : 'y' + str(i)})
                # temp = pd.concat([temp, pd.concat([hv_geocoords, lv_geocoords], ignore_index=True)], axis=1)
                #i += 1
            #temp = temp.drop(['coords'], axis=1)
        input_geoCoords = temp
        #return input_geoCoords

    input_geoJSON = {"type" : "FeatureCollection", "features": []}

    for point in input_geoCoords.T:
        currentFeatureProperties = {}
        index = 0
        if featureProperties:
            index = point if (featureName == 'bus' or featureName == 'line') else input_data.index[point]
            currentFeatureProperties["index"] = index
            
            for property in featureProperties:
                if property in input_data:
                    currentFeatureProperties[property] = input_data.loc[index][property]
                else:
                    currentFeatureProperties[property] = ""

        if propertyGroupNames and propertyGroupFeatures:
            for property in propertyGroupNames:
                if property in ppdata:
                    try:
                        propertyGroup = ppdata[property].loc[ppdata[property].bus == index]
                    except KeyError:
                        propertyGroup = pd.DataFrame()
                extractedProperties = extractPropertiesFromNet(propertyGroup, index, propertyGroupFeatures[propertyGroupNames.index(property)])
                currentFeatureProperties[property] = extractedProperties
        #return 0
        inputCoordinates = input_geoCoords.loc[point].tolist()    
        feature = { "type": "Feature", 
                        "geometry": {"type": "LineString" if isLines else "Point", 
                                     "coordinates": inputCoordinates}, 
                        "properties": currentFeatureProperties
                    }
        input_geoJSON["features"].append(feature)
    return input_geoJSON

In [53]:
input_data = createFeatures(False, testnet, 'bus', bus_properties, ['load', 'sgen', 'switch'], [load_features, sgen_features, switch_features])
input_data

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [7.765225672614365, 48.41091584192147]},
 'properties': {'index': 0,
  'name': 'Bus 0',
  'vn_kv': 20.0,
  'type': 'n',
  'in_service': True,
  'max_vm_pu': '',
  'min_vm_pu': '',
  'load': {'name': 'LV Load 67',
   'bus': 0,
   'p_mw': 0.4,
   'q_mvar': 0.08122346425360166,
   'const_z_percent': 0.0,
   'const_i_percent': 0.0,
   'sn_mva': nan,
   'scaling': 0.6,
   'in_service': True,
   'type': 'MV/LV Station'},
  'sgen': {'name': 'Static Generator 0',
   'bus': 0,
   'p_mw': 0.12215,
   'q_mvar': 0.0,
   'sn_mva': 0.12215,
   'scaling': 0.0,
   'in_service': True,
   'type': 'PV',
   'current_source': True},
  'switch': {'bus': 0,
   'element': 130,
   'et': 'l',
   'type': 'LBS',
   'closed': True,
   'name': 'Switch 221',
   'z_ohm': 0.0,
   'in_ka': nan}}}

In [54]:
input_data = createFeatures(True, testnet, 'trafo', trafo_properties, '', '')
input_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'LineString',
    'coordinates': [[7.913960629334794, 48.4569382040042],
     [7.913960629334794, 48.4569382040042]]},
   'properties': {'index': 114,
    'name': 'HV/MV Transformer 0',
    'hv_bus': 58,
    'lv_bus': 39,
    'vk0_percent': '',
    'vkr0_percent': '',
    'mag0_percent': '',
    'mag0_rx': '',
    'si0_hv_partial': '',
    'tap_pos': -2,
    'in_service': True,
    'max_loading_percent': '',
    'parallel': 1,
    'df': 1.0,
    'tap_dependent_impedance': '',
    'vk_percent_characteristic': '',
    'vkr_percent_characteristic': '',
    'xn_ohm': '',
    'std_type': '25 MVA 110/20 kV'}},
  {'type': 'Feature',
   'geometry': {'type': 'LineString',
    'coordinates': [[7.7615333943715035, 48.439874883455424],
     [7.7615333943715035, 48.439874883455424]]},
   'properties': {'index': 142,
    'name': 'HV/MV Transformer 1',
    'hv_bus': 318,
    'lv_bus': 319,
    'vk0_percent': '',
 

In [55]:
input_data = createFeatures(False, testnet, 'ext_grid', ext_grid_properties, '', '')
input_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [7.913960629334794, 48.4569382040042]},
   'properties': {'index': 0,
    'name': 'External Grid 0',
    'bus': 58,
    'vm_pu': 1.0,
    'va_degree': 0.0,
    's_sc_max_mva': nan,
    's_sc_min_mva': nan,
    'rx_max': nan,
    'rx_min': nan,
    'max_p_mw': '',
    'max_q_mvar': '',
    'min_q_mvar': '',
    'r0x0_max': '',
    'x0x_max': '',
    'slack_weight': 1.0,
    'controllable': '',
    'in_service': True}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [7.7615333943715035, 48.439874883455424]},
   'properties': {'index': 1,
    'name': 'External Grid 1',
    'bus': 318,
    'vm_pu': 1.0,
    'va_degree': 0.0,
    's_sc_max_mva': nan,
    's_sc_min_mva': nan,
    'rx_max': nan,
    'rx_min': nan,
    'max_p_mw': '',
    'max_q_mvar': '',
    'min_q_mvar': '',
    'r0x0_max': '',
    'x0x_max': '',
    'slack_weight': 1.0,
    'control

In [56]:
input_data = createFeatures(True, testnet, 'line', line_properties, '', '')
input_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'LineString',
    'coordinates': [[[7.8947079593416, 48.40549007606241],
      [7.896048283667894, 48.41060722903666],
      [7.896173712216692, 48.41100311474432]]]},
   'properties': {'index': 0,
    'name': 'Line 0',
    'from_bus': 238,
    'to_bus': 109,
    'length_km': 0.5863662,
    'r0_ohm_per_km': '',
    'x0_ohm_per_km': '',
    'c0_nf_per_km': '',
    'df': 1.0,
    'g_us_per_km': 0.0,
    'g0_us_per_km': '',
    'parallel': 1,
    'max_loading_percent': '',
    'temperature_degree_celsius': '',
    'tdpf': '',
    'wind_speed_m_per_s': '',
    'wind_angle_degree': '',
    'conductor_outer_diameter_m': '',
    'air_temperature_degree_celsius': '',
    'reference_temperature_degree_celsius': '',
    'solar_radiation_w_per_sq_m': '',
    'solar_absorptivity': '',
    'emissivity': '',
    'r_theta_kelvin_per_mw': '',
    'mc_joule_per_m_k': '',
    'std_type': 'NA2XS2Y 1x185 RM/25 12/20 kV